### Importing necessary libraries

In [ ]:
from glob import glob
import json
from pprint import pprint
import pandas as pd
import geopandas as gpd
import rasterio
import numpy as np
import folium
import rioxarray as riox
from datetime import datetime, timedelta
from rasterio.mask import mask
from shapely.geometry import Point


##### STEP 1 AOI and coordinates
Finding coordinates for the Area of Interest (AOI) based on the defined .shp in geographical WGS84 reference frame (EPSG4326). This reference system was used due to the names used for the identification of RTC products (RTC products are named and are available in geographic NorthEeast schema). Therefore it is recommended to use .shp in EPSG4326 in this step. Afterwards, within the further processing steps. The products will be converted and produced in EPSG32631 reference system. In futher steps we will, work on the automatic reprojection of shp witin the Jupyter Notebook (TO BE DONE!)
If you want to check min/max coordinates of the interested AOI call "coordinates" command,
* to define the path to the shp change "shp_path" remember about the slash (/)

In [ ]:
actual_path = os.getcwd()
classification_path = actual_path + ""
shp_path = classification_path + ""

shp = gpd.read_file(shp_path)

coordinates = shp.total_bounds

min_x = int(coordinates[0])
min_y = int(coordinates[1])
max_x = int(coordinates[2])
max_y = int(coordinates[3])

for x in range(min_x, max_x + 1):
    for y in range(min_y, max_y + 1):
        coordinate = f"N{y:02d}E{x:03d}"
        print(coordinate)
        
def read_shapefile(filepath):
    sf = shapefile.Reader(filepath)
    shapes = sf.shapes()
    records = sf.records()
    fields = sf.fields


#### STEP 2 Searching for the RTC products based on AOI
* Searching available Radiometrically Terrain Corrected (RTC) products based on the previously identified max/min coordinates of AOI
* Here, please specify date in "time".Note that ** defines all available images in the month. Here we define all products for January 2023
* Notice the appropriate slash usage !
* Coordinates of Lagos N06E003
* Available products for AOI and selected "Time" are saved afterwards to .txt file named available_RTCproducts_for_AOI

In [ ]:
path = ""
time = ""

for x in range(min_x, max_x + 1):
    for y_value in range(min_y, max_y + 1):
        coordinate = f"N{y_value:02d}E{x:03d}"
        r = coordinate

        files = glob.glob(path + time + r + '*')

        with open("", mode='a') as f:
            for file in files:
                f.write(file + '\n')
def count_line(filename):
    with open(filename, 'r') as file:
        lines = sum(1 for line in file)
    return lines

filename = ""
lines = count_line(filename)
print(f'Number of product found for thwe selected AOI: {lines}')

##### Optional: Display metada from selected image
* Add number, and modify variables
* Optional

In [ ]:
json_text = str()
date='/2023/01/15/'
dat='_2023_01_15_'
coord='N06E002'
number='059C5A'
e='metadata.json'
with open(p+date+coord+dat+number+'/s1_rtc_'+number+'_'+coord+dat+ e, 'r') as f:
    json_text = f.read()

data = json.loads(json_text)


#### STEP 3 Accessing RTC products metadata
* Making paths with names and ends 'metada.json' to access metadeta of the identified RTC products (geometry, relative orbit number etc.)
* If you want to print the path to metadata of each product uncommend the #" part of code

In [ ]:
file_path = ""
extension = 'metadata.json'

with open(file_path, 'r') as file:
    paths = [line.strip() for line in file]

files = []
for path in paths:
    files.extend(glob.glob(path + '/*' + extension))
    

##### Storing products in dataframe
* Here all idenfied RTC products are stored in one Dataframe with path, orbits, mode 

In [ ]:
orbits = []  
names = []
datetime = []  
mode = []  
nor= []
sao=[]
for file in files:
    with open(file, 'r') as f:
        json_text = f.read()

    data = json.loads(json_text)
    
    names.append(data['id'])
    orbits.append(data['properties']['sat:orbit_state'])
    datetime.append(data['properties']['datetime']) 
    nor.append(data['properties']['sat:relative_orbit'])

df = pd.DataFrame(list(zip(files, names, orbits, datetime, nor,)), columns=['Path', 'Name', 'Orbit', 'Datetime', 'Relative_orbit'])
new = df["Datetime"].str.split("T", n = 1, expand = True)
df["Date"]= new[0]
df["Time"]= new[1]
df.drop(['Datetime'], axis=1)
df.to_csv("" +"", sep=',', index=False, header=True)
df.head(5)

##### STEP 5 Finding appropriate Relative Orbit number for AOI
* Here, we can found number of available RTC images within AOI for each specific RelativeOrbitNumber. There are some cases, when only few products are found for the AOI which are not covered the AOI and then merging of RTC products from various relative orbit numbers and various dates will be needed. This is also check for TO DO LIST! At the moment, we toolk for the investigation, the relative orbit number for which we found the most available products (here RelativeOrbitNumber =1) 

In [ ]:
stat = []
unique=df['Relative_orbit'].unique()
for i in unique:
    no=len(df[df.Relative_orbit== i])
    stat = stat + [no]
stat = np.array(stat)
combined = np.vstack((unique, stat)).T
df1 = pd.DataFrame(combined)
df1.columns=['RelativeOrbitNumber','no_of_found_images_within_AOI']
df1

### Selecting appropriate dates, please specify relative orbit number which you select in "df_RO= df[df['Relative_orbit'] ==1]"

In [ ]:
stat = []
df_RO= df[df['Relative_orbit'] ==1]
unique=df_RO['Date'].unique()
for i in unique:
    no=len(df_RO[df_RO.Date== i])
    stat = stat + [no]
stat = np.array(stat)
combined = np.vstack((unique, stat)).T
df2 = pd.DataFrame(combined)
df2.columns=['Date','no_of_found_images_within_AOI']
maximum_o_images=max(df2['no_of_found_images_within_AOI'])
df2

### STEP 6 Selection of the appropriate Relative Orbit Number and saving selected path with VV and VH extenstion to csv

In [ ]:
from datetime import datetime
actual_path=os.getcwd()
klasyfikacja_path = ""
df_selectedproducts=df[df['Relative_orbit'] ==1]
df_sliceVV= df_selectedproducts.loc[:, ('Path','Date')]
df_sliceVV['Path'] = df_sliceVV['Path'].str.replace('metadata.json', '')
df_sliceVV['Path'] =df_sliceVV['Path'].astype(str) +  'VV.tif'

pd.to_datetime(df_sliceVV['Date'], format='%Y-%m-%d')
df_sliceVV['date'] = pd.to_datetime(df_sliceVV['Date'], format='%Y-%m-%d')
df_sliceVV.to_csv("" +"", sep=',', index=False, header=True)

df_sliceVH= df_selectedproducts.loc[:, ('Path','Date')]
df_sliceVH['Path'] = df_sliceVH['Path'].str.replace('metadata.json', '')
df_sliceVH['Path'] = df_sliceVH['Path'].astype(str) +  'VH.tif'
pd.to_datetime(df_sliceVH['Date'], format='%Y-%m-%d')
df_sliceVH['date'] = pd.to_datetime(df_sliceVH['Date'], format='%Y-%m-%d')
df_sliceVH.to_csv("" +"", sep=',', index=False, header=True)



### Creates a folder where merged RTC products will be stores folder name "merged_RTC"

In [ ]:
actual_path=os.getcwd()
klasyfikacja_path = actual_path+""
merged_path = actual_path+""+""
isExist = os.path.exists(merged_path)
if not isExist:
    os.makedirs(merged_path)
    print("created folder: {}".format(merged_path))
print("Folder already exists")

### Display tiff and shp on the map

In [ ]:
file_paths = [""]
shp_path = ""

gdf = gpd.read_file(shp_path)

emap = folium.Map(location=[gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()], zoom_start=8)

folium.GeoJson(gdf).add_to(emap)

for file_path in file_paths:
    image_overlay = folium.raster_layers.ImageOverlay(
        image=file_path,
        bounds=[[6, 2], [8, 5]],
        opacity=1,
        name="Image Overlay"
    )
    image_overlay.add_to(emap)
emap